# Predict the year from the abstract 

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from os.path import exists

import string # библиотека для работы со строками
import nltk   # Natural Language Toolkit
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Binarizer

In [ ]:
from data_types import AuthorsDB
from data_types import PublicationsDB
from data_types import AbstractsDB

audb = AuthorsDB()
audb.load()
pubdb = PublicationsDB()
pubdb.load()
absdb = AbstractsDB()
absdb.load()

filename = "../data/mnid_author_dict.pkl"
with open(filename,'rb') as inp:
    authors_dict = pickle.load(inp)

In [ ]:
# authors_dict2 = {v['mn_id']:v['name'] for k,v in authors_dict.items()}

In [ ]:
ind = 0
for k, v in authors_dict.items():
    ind += 1
    print(k,v)
    if ind>4:
        break
        

In [ ]:
# filename = '../data/mnid_author_dict.pkl'    
# with open(filename,'wb') as outp:
#     pickle.dump(authors_dict2, outp, pickle.HIGHEST_PROTOCOL)

In [ ]:
# filename = "../data/mathnet_iam_authors_dict.pkl"
# with open(filename,'rb') as inp:
#     authors_dict = pickle.load(inp)

In [ ]:
pub = pd.DataFrame.from_dict(pubdb.db,orient='index')

# 1 Data Preprocessing

I need to get 
- abstract and it's tokenization
- year of paper



In [ ]:
abstracts = absdb.db[absdb.db['abstract'].notna()].copy()

In [ ]:
absdb.db.info()

In [ ]:
abstracts.info()

In [ ]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

# примеры стоп-слов
stop_words[:10]

In [ ]:
# знаки препинания
string.punctuation

In [ ]:
word_tokenizer = nltk.WordPunctTokenizer()

In [ ]:
# dates = [str(x) for x in np.arange(1900, 2022)]


def process_abstractsdb(data):
    texts = []    
    
    # поочередно проходим по всем новостям в списке
    for index,row in data.iterrows():
        # print(row['abstract'])        
        # print(type(row['abstract']))
        text = row['abstract']
        if (row['keywords'] is not None):
            # print(text)
            # print(row['keywords'])
            # print(type(row['keywords']))
            text += row['keywords']                   
        text_lower = text.lower() # приводим все слова к нижнему регистру
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем екст на слова                
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and not word.isnumeric())]
                      
        texts.append(tokens) # добавляем в предобработанный список
    
    return texts

In [ ]:
abstracts.loc[['vyurv213','vyurv1','vyurv46']].info()

In [ ]:
# texts = process_abstractsdb(abstracts.loc[['vyurv213','vyurv1','vyurv46']])
# texts = process_abstractsdb(abstracts.loc[['vyurv213','vyurv1','vyurv46']])
texts = process_abstractsdb(abstracts)

In [ ]:
texts

In [ ]:
# инициализируем лемматизатор :)
morph = pymorphy2.MorphAnalyzer()

In [ ]:
for aword in texts[2]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s \tЛемматизированное: %s" % (aword, aword_norm))

In [ ]:
# применяем лемматизацию ко всем текстам
print(range(len(texts)))

file_lemmatized_text = '../data/lemmatized_text.pkl'
if exists(file_lemmatized_text):    
    with open(file_lemmatized_text,'rb') as inp:
        texts = pickle.load(inp)
else:
    for i in tqdm(range(len(texts))):           # tqdm_notebook создает шкалу прогресса :)
        text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
        texts[i] = ' '.join(text_lemmatized)
    # time.sleep(1)  

In [ ]:
def save_lemmatized_text(data, status='write'):
    filename = '../data/lemmatized_text.pkl'    
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)
# save_lemmatized_text(texts)

In [ ]:
# texts

https://stackoverflow.com/questions/46118910/scikit-learn-vectorizer-max-features

In [ ]:
# Fit TF-IDF on train texts
# vectorizer = TfidfVectorizer(min_df=1,   max_features=None, strip_accents='unicode',  analyzer='word',token_pattern=r'\w{2,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)
# vectorizer = TfidfVectorizer(min_df=1,   max_features=None, analyzer='word',token_pattern=r'\w{2,}', ngram_range=(1, 1), use_idf=1,smooth_idf=1,sublinear_tf=1) 
my_min_df = 0.0002
vectorizer_uni = TfidfVectorizer(min_df=1,   max_features=None, analyzer='word',token_pattern=r'\w{2,}', ngram_range=(1, 1), use_idf=1,smooth_idf=1,sublinear_tf=1) 
vectorizer_bi = TfidfVectorizer(min_df=my_min_df,   max_features=None, analyzer='word',token_pattern=r'\w{2,}', ngram_range=(2, 2), use_idf=1,smooth_idf=1,sublinear_tf=1) 
vectorizer_tri = TfidfVectorizer(min_df=my_min_df,   max_features=None, analyzer='word',token_pattern=r'\w{2,}', ngram_range=(3, 3), use_idf=1,smooth_idf=1,sublinear_tf=1) 
vectorizer_uni.fit(texts)
vectorizer_bi.fit(texts)
vectorizer_tri.fit(texts)


In [ ]:
print('uni:', len(vectorizer_uni.get_feature_names_out()))
print('bi:',len(vectorizer_bi.get_feature_names_out()))
print('tri:',len(vectorizer_tri.get_feature_names_out()))

In [ ]:
max_val = 100
for ind,item in enumerate(vectorizer_uni.get_feature_names_out()):
    print(ind, item)
    if ind >max_val:
        break

for ind,item in enumerate(vectorizer_bi.get_feature_names_out()):
    print(ind, item)
    if ind >max_val:
        break
    
for ind,item in enumerate(vectorizer_tri.get_feature_names_out()):
    print(ind, item)
    if ind >max_val:
        break    

In [ ]:
tf_idf_matrix_uni = vectorizer_uni.fit_transform(texts)
tf_idf_matrix_bi = vectorizer_bi.fit_transform(texts)
tf_idf_matrix_tri = vectorizer_tri.fit_transform(texts)

In [ ]:
print(type(tf_idf_matrix_uni))
print(tf_idf_matrix_uni.shape)

In [ ]:
def save_vectorizer(data,name,status='write'):
    filename = '../data/'+name+'.pkl'    
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)
            
def save_tf_idf_matrix(data,name,status='write'):
    filename = '../data/'+name+'.pkl'    
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)

In [ ]:
# save_vectorizer(vectorizer_uni,'vectorizer_uni',status="write")
# save_vectorizer(vectorizer_bi,'vectorizer_bi',status="write")
# save_vectorizer(vectorizer_tri,'vectorizer_tri',status="write")
# save_tf_idf_matrix(tf_idf_matrix_uni,'tf_idf_matrix_uni',status="write")
# save_tf_idf_matrix(tf_idf_matrix_bi,'tf_idf_matrix_bi',status="write")
# save_tf_idf_matrix(tf_idf_matrix_tri,'tf_idf_matrix_tri',status="write")

In [ ]:
def load_vectorizer(name):
    filename = '../data/'+name+'.pkl'
    res = None    
    with open(filename,'rb') as inp:
        res = pickle.load(inp)
    return res

def load_tf_idf_matrix(name):
    filename = '../data/'+name+'.pkl'
    res = None    
    with open(filename,'rb') as inp:
        res = pickle.load(inp)
    return res


In [ ]:
# x = load_vectorizer()

In [ ]:
# x.get_feature_names_out()[:10]

# Importance of the article for Request

In [ ]:
def process_request(data):        
    text_lower = data.lower()
    tokens = word_tokenizer.tokenize(text_lower)
    tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and not word.isnumeric())]        
    texts = ' '.join(tokens)         
    return [morph.parse(texts)[0].normal_form]

In [ ]:
from collections import defaultdict
def get_authors_rating(df, coef):
    # print(df)
    # print(type(df))
    # print(coef)
    # print(type(coef))
    rating = defaultdict(int)
    
    for index in range(len(df)):
        # print(f'index = {index}, row = {df.iloc[index]}')        
        for author in df.iloc[index]['author_id']:  
            # print(index)          
            # print(f'add coef = {coef[index]} to author = {author}')
            rating[author] += coef[index]            
            # print(rating)
    # dict(sorted(rating.items(), key=lambda item: item[1]))
    rating_list = [ (k,v) for k, v in sorted(rating.items(), key=lambda item: item[1], reverse=True)]
    return rating_list

In [ ]:


# request = process_request(request)
# print(request)
# print(type(request))



In [ ]:
# result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)
def get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict):    
    request = process_request(request)    
    vect_req = vectorizer.transform(request)
    onehot = Binarizer()
    ohe_request = onehot.fit_transform(vect_req.toarray())    
    coef= tf_idf_matrix.dot(ohe_request.T)    
    coef = coef.flatten()    
    
    max_number_of_articles = 30
    low_bound = 0.01
    max_output_pubs = 5
    max_output_authors = 5
    
    pubs_index = np.argsort(coef)[::-1][:max_number_of_articles]        
    low_bound_pub_index  = [ind for ind in pubs_index if coef[ind]>low_bound]
    req_pups = list(abstracts.iloc[low_bound_pub_index].index)    
    top_pubs = pub.loc[req_pups]['reference']
    top_autors = get_authors_rating(pub.loc[req_pups],coef[pubs_index])
        
    index = 0
    for item in top_autors:        
        if index >= max_output_authors:
            break
        if item[0] in authors_dict:
            index +=1
            print(f'author: {authors_dict[item[0]]}, raiting: {round(item[1],2)}')            
            
    for item in top_pubs[:max_output_pubs]:
        print(item)    

In [ ]:
def get_model_response_print(request, vectorizer, tf_idf_matrix, abstracts, authors_dict):
    # tf_idf_matrix= train_X = vectorizer.fit_transform(texts)
    # print(request)
    # print(type(request))
    request = process_request(request)
    # print(request)
    # print(type(request))
    vect_req = vectorizer.transform(request)
    onehot = Binarizer()
    ohe_request = onehot.fit_transform(vect_req.toarray())
    # print(ohe_request.shape)
    # print(tf_idf_matrix.shape)
    coef= tf_idf_matrix.dot(ohe_request.T)
    # print(coef)
    coef = coef.flatten()
    
    
    max_number_of_articles = 30
    low_bound = 0.01
    max_output_pubs = 5
    max_output_authors = 5
    
    pubs_index = np.argsort(coef)[::-1][:max_number_of_articles]
    # print(pubs_index)
    # coef[pubs_index]
    
    low_bound_pub_index  = [ind for ind in pubs_index if coef[ind]>low_bound]
    # print(low_bound_pub_index)
    # print(coef[low_bound_pub_index])
    
    req_pups = list(abstracts.iloc[low_bound_pub_index].index)
    print(req_pups)
    print(pub.loc[req_pups].head())
    
    top_pubs = pub.loc[req_pups]['reference']
    
    top_autors = get_authors_rating(pub.loc[req_pups],coef[pubs_index])
    print(top_autors[:max_output_authors][0], top_autors[:max_output_authors][1])
    print(type(top_autors))
    print(top_autors)
    index = 0
    for item in top_autors:
        print(index, item)        
        if index >= max_output_authors:
            break
        if item[0] in authors_dict:
            index +=1
            print(f'author = {authors_dict[item[0]]}, his raiting = {round(item[1],2)}')
            
            
    for item in top_pubs[:max_output_pubs]:
        print(item)

In [ ]:
request = 'суперкомпьютерное моделирование отказоустойчивые вычисления на hpc системах'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'суперкомпьютерное'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'моделирование'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'отказоустойчивые вычисления на hpc системах'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'отказоустойчивые вычисления'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'отказоустойчивые'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'ULFM'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'вычисления'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'интеллект'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'искусственный'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

In [ ]:
request = 'искусственный интеллект'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)